In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/fastai-practice

# !git config --global user.email 'intaek428@gmail.com'
# !git config --global user.name 'ITHwang'

# !git add .
# !git commit -m ''
# !git push

!pip install -Uqq fastbook

In [ ]:
from fastai.text.all import *

path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])

In [ ]:
files

(#100000) [Path('/root/.fastai/data/imdb/train/neg/9467_1.txt'),Path('/root/.fastai/data/imdb/train/neg/6960_1.txt'),Path('/root/.fastai/data/imdb/train/neg/3812_1.txt'),Path('/root/.fastai/data/imdb/train/neg/11328_3.txt'),Path('/root/.fastai/data/imdb/train/neg/6758_4.txt'),Path('/root/.fastai/data/imdb/train/neg/1216_3.txt'),Path('/root/.fastai/data/imdb/train/neg/7126_1.txt'),Path('/root/.fastai/data/imdb/train/neg/7488_1.txt'),Path('/root/.fastai/data/imdb/train/neg/5014_1.txt'),Path('/root/.fastai/data/imdb/train/neg/615_1.txt')...]

In [ ]:
txt = files[0].open().read()
txt[:75]

'If you liked the Grinch movie... go watch that again, because this was no w'

In [ ]:
txt

'If you liked the Grinch movie... go watch that again, because this was no where near as good a Seussian movie translation. Mike Myers\' Cat is probably the most annoying character to "grace" the screen in recent times. His voice/accent is terrible and he laughs at his own jokes with an awful weasing sound, which is about the only laughing I heard at the theater. Not even the kids liked this one folks, and kids laugh at anything now. Save your money and go see Looney Tunes: Back in Action if you\'re really looking for a fun holiday family movie.'

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
coll_repr(toks, 30)

'(#116) [\'If\',\'you\',\'liked\',\'the\',\'Grinch\',\'movie\',\'...\',\'go\',\'watch\',\'that\',\'again\',\',\',\'because\',\'this\',\'was\',\'no\',\'where\',\'near\',\'as\',\'good\',\'a\',\'Seussian\',\'movie\',\'translation\',\'.\',\'Mike\',\'Myers\',"\'",\'Cat\',\'is\'...]'

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
subword(1000)

'▁I f ▁you ▁like d ▁the ▁G ri n ch ▁movie ... ▁go ▁watch ▁that ▁again , ▁because ▁this ▁was ▁no ▁where ▁near ▁as ▁good ▁a ▁S e us s ian ▁movie ▁trans l ation . ▁M i ke ▁M'

In [ ]:
type(txts)

fastcore.foundation.L

In [ ]:
tkn = Tokenizer(spacy)
toks200 = txts[:200].map(tkn)
toks200[0]

(#130) ['xxbos','xxmaj','if','you','liked','the','xxmaj','grinch','movie','…'...]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2000) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','i','in'...]"

In [ ]:
num.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 '.',
 ',',
 'a',
 'and',
 'of',
 'to',
 'is',
 'it',
 'i',
 'in',
 'this',
 'that',
 "'s",
 '"',
 'was',
 '-',
 'movie',
 '\n\n',
 'for',
 'with',
 'as',
 'but',
 ')',
 '(',
 'you',
 "n't",
 'film',
 'not',
 'on',
 'have',
 'are',
 'be',
 '!',
 'all',
 'he',
 'at',
 'by',
 'one',
 'his',
 '?',
 'just',
 'they',
 'if',
 'like',
 'an',
 'so',
 'there',
 'or',
 'who',
 'from',
 '…',
 'do',
 'out',
 'about',
 'her',
 'has',
 "'",
 'no',
 'good',
 'even',
 'what',
 'she',
 'up',
 'bad',
 'would',
 'some',
 'more',
 'my',
 'did',
 'time',
 'can',
 'me',
 'only',
 'were',
 'when',
 'had',
 'see',
 'does',
 'really',
 'could',
 'much',
 'get',
 'very',
 'than',
 'make',
 'well',
 ':',
 'people',
 'made',
 'which',
 'acting',
 '&',
 'plot',
 'we',
 'any',
 'first',
 'their',
 'story',
 'how',
 'into',
 'them',
 'off',
 'because',
 ';',
 'nothing',
 'your',
 'will',
 'way',
 'been',
 'other',
 'why

In [ ]:
nums = num(toks)[:20]
nums

TensorText([  0,  34, 614,   9,   0,  26,   0, 168, 141,  21, 203,  11, 112,  20,
         24,  67, 132, 478,  30,  68])

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxunk you liked the xxunk movie xxunk go watch that again , because this was no where near as good'

In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)
x, y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj if you liked the xxmaj xxunk movie … go watch that again , because this was no where'

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj if you liked the xxmaj xxunk movie … go watch that again , because this was no where near'

In [ ]:
from functools import partial

get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1),
).dataloaders(path, path=path, bs=16, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj john xxmaj hughes wrote a lot of great comedies in the ' 80s . "" european xxmaj vacation "" is not one of them . xxmaj the follow - up to xxmaj hughes ' first big hit "" vacation "" ( 1983 ) , is about as predictable , unfunny and annoying as they come -- no matter how much you love the dumb but romantic xxmaj clark and xxmaj ellen xxmaj griswold ( chase and d'angelo )","xxmaj john xxmaj hughes wrote a lot of great comedies in the ' 80s . "" european xxmaj vacation "" is not one of them . xxmaj the follow - up to xxmaj hughes ' first big hit "" vacation "" ( 1983 ) , is about as predictable , unfunny and annoying as they come -- no matter how much you love the dumb but romantic xxmaj clark and xxmaj ellen xxmaj griswold ( chase and d'angelo ) ."
1,", 100 xxmaj songs for , the film was number 23 on 100 xxmaj years of xxmaj musicals , and it was number 36 on xxmaj the 100 xxmaj greatest xxmaj musicals . xxmaj very good ! xxbos a bunch of sorority girls make a new pledge spend the night in a creepy mausoleum . xxmaj of course the recently deceased do n't stay deceased for long and all hell breaks loose . xxunk xxmaj dark xxmaj night "" is","100 xxmaj songs for , the film was number 23 on 100 xxmaj years of xxmaj musicals , and it was number 36 on xxmaj the 100 xxmaj greatest xxmaj musicals . xxmaj very good ! xxbos a bunch of sorority girls make a new pledge spend the night in a creepy mausoleum . xxmaj of course the recently deceased do n't stay deceased for long and all hell breaks loose . xxunk xxmaj dark xxmaj night "" is an"


In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: ignored